In [3]:
# Libraries
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, PromptTuningConfig, AdaLoraConfig, TaskType, PeftConfig
import torch
import os 
import seaborn as sns
import matplotlib.pyplot as plt


#Path
path = "C:/Users/diego/Desktop/Master Neuro/M2/Intership_NLP/multimodal-argmining"
os.chdir(path)

#Model Name
MODEL_NAME = "roberta-base"

In [4]:
#Load Dataset
train_path = f"{path}/data/gun_control_train.csv"
dev_path   = f"{path}/data/gun_control_dev.csv"
test_path  = f"{path}/data/gun_control_test.csv"

df_train = pd.read_csv(train_path)
df_dev   = pd.read_csv(dev_path)
df_test  = pd.read_csv(test_path)


# Map labels to ints
label2id = {"oppose": 0, "support": 1}
for df in [df_train, df_dev, df_test]:
    df["label"] = df["stance"].map(label2id)

print(df_train["label"].value_counts())
df_train.head()

label
1    475
0    448
Name: count, dtype: int64


,tweet_id,tweet_url,tweet_text,stance,persuasiveness,split,label
0,1372936384034447366,https://t.co/FpkVZ8ESy0,More Asian-Americans Are Buying Guns For Prote...,oppose,no,train,0
1,1327310308260667393,https://t.co/KrJTpJR3Ke,"""I will protect your Bill of Rights. Gun contr...",oppose,no,train,0
2,1334523148692312065,https://t.co/hBAV1pPCY9,#guns #2A 6-Time Olympic Shooting Medalist Say...,oppose,no,train,0
3,1324087921641721856,https://t.co/LfIzR6iPA3,Congratulations @ForHD65 on your victory! \n\n...,support,no,train,1
4,1313162243035607040,https://t.co/MZyeIP6Mtx,Dr. Cindy Banyai supports common sense gun saf...,support,no,train,1


In [5]:
#Tokenization
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#Tokenization per Batch
def tokenize_batch(batch):
    return tokenizer(batch["tweet_text"],padding="max_length",truncation=True,max_length=128)


dataset_train = Dataset.from_pandas(df_train[["tweet_text", "label"]])
dataset_dev   = Dataset.from_pandas(df_dev[["tweet_text", "label"]])
dataset_test  = Dataset.from_pandas(df_test[["tweet_text", "label"]])

dataset_train = dataset_train.map(tokenize_batch, batched=True)
dataset_dev   = dataset_dev.map(tokenize_batch, batched=True)
dataset_test  = dataset_test.map(tokenize_batch, batched=True)

Map: 100%|██████████| 150/150 [00:00<00:00, 2836.11 examples/s]


### LoRA

In [6]:
# Configure Lore
lora_config = LoraConfig(
    task_type="SEQ_CLS",  # Secuencia de clasificación
    r=8,                  # Dimensión baja del adaptador
    lora_alpha=32,        # Escala
    lora_dropout=0.1,     # Dropout para regularización
    target_modules=["query", "value"],  # Solo aplicamos LoRA a Q y V en self-attention,
    bias="none"
)

#### Parameters


| Parameter | Description | How to Choose / Rule of Thumb |
|-----------|-------------|-------------------------------|
| **task_type** | Type of task the model is fine-tuned for (e.g., sequence classification, generation). | `"SEQ_CLS"` for classification, `"SEQ_2_SEQ_LM"` for generation, `"CAUSAL_LM"` for causal decoders. |
| **r** | Low-rank dimension of the LoRA matrices (controls capacity). | Small models: 4–16; Large models: 16–64. Higher → more expressive but more parameters. |
| **lora_alpha** | Scaling factor for the LoRA update \(W + αBA\). | Usually 1–4 × `r`. Too low → weak updates; too high → unstable training. |
| **lora_dropout** | Dropout applied to the LoRA module for regularization. | Small datasets: 0.1–0.2; Large datasets: 0–0.1. Prevents overfitting. |
| **target_modules** | Specifies which layers are modified with LoRA. | Common: `["query", "value"]` for attention. Can include feed-forward (`"dense"`) or `"all"`. |
| **fan_in_fan_out** | Adjusts matrix orientation; required for some architectures like GPT. | Usually left as default unless the model needs it. |
| **merge_weights** | Whether to merge LoRA weights into the base model after training. | Merge after fine-tuning to reduce memory usage. |
| **bias** | Whether LoRA affects biases or only weight matrices. | Usually keep bias unchanged. |


### PromptTuning

In [7]:
prompt_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12)


#### Parameters
| Parameter | Description | How to Choose / Rule of Thumb |
|-----------|-------------|-------------------------------|
| task_type | Type of task | SEQ_CLS for classification |
| num_virtual_tokens | Number of learnable prompt tokens | 10–50 depending on task complexity |
| encoder_hidden_size | Dimensionality of model embeddings | Usually same as model hidden size |
| prompt_dropout | Regularization | 0–0.1 |


### AdaLoRA

In [8]:
adalora_config =AdaLoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.01,
    tinit=0.05,
    tfinal=0.01,
    total_step=171 # ((num_train_Examples // Batch_size ))*Epochs
)

#### Parameters
| Parameter | Description | How to Choose / Rule of Thumb |
|-----------|-------------|-------------------------------|
| task_type | Type of task | SEQ_CLS for classification |
| r | Initial low-rank dimension | Small models: 4–16 |
| target_modules | Layers to adapt | Common: ["query","value"] |
| rank_dropout | Dropout applied to rank | 0–0.1 |
| tinit | Initial importance threshold | 0.01–0.1 |
| tfinal | Final threshold for pruning | 0.001–0.01 |

In [9]:
#Auxiliar Function to get the % of the parameters we will train per each PEFT
def count_trainable_params(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    percent = 100 * trainable / total
    return total, trainable, percent

In [10]:
# Define General Training Arguments and a For Loop for future Benchark 

def train_peft(peft_type,peft_config):

    training_args = TrainingArguments(
        output_dir=f".experiments/PEFT/Text_Model/results_{peft_type}",
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir=f"./logs_{peft_type}",
        logging_steps=50,
        load_best_model_at_end=True,
        fp16=True
    )

    #Model Base
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

    total_params, trainable_params, percent_trainable = count_trainable_params(model)
    print(f"Trainable params: {trainable_params:,} / {total_params:,} ({percent_trainable:.2f}%)")


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_dev,
        tokenizer=tokenizer
    )

    trainer.train()
    eval_metrics = trainer.evaluate(dataset_test)
    return eval_metrics


In [ ]:
#Train

peft_configs = {
    "LoRa": lora_config,
    "PromptTuning": prompt_config,
    "AdaLoRA": adalora_config
}

results={}
for peft,config in peft_configs.items():
    print(f"\n Training with {peft}")
    metrics=train_peft(peft,config)
    results[peft]=metrics


 Training with LoRa


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066
Trainable params: 887,042 / 125,534,212 (0.71%)


C:\Users\diego\AppData\Local\Temp\ipykernel_8260\1938471710.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\diego\anaconda3\envs\multimodal\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
# Convert results to DataFrame
df_results = pd.DataFrame(results).T
df_results = df_results.reset_index().rename(columns={"index": "PEFT_Type"})

display(df_results[["PEFT_Type", "eval_loss", "eval_accuracy"]])

sns.barplot(data=df_results, x="PEFT_Type", y="eval_accuracy")
plt.title("Accuracy Comparison — LoRA vs Adapter vs BitFit")
plt.show()

### Conclusions

- **LoRA**: Best balance between performance and efficiency. Injects low-rank matrices into attention layers, allowing the model to adapt quickly with few parameters.
- **Prompt Tuning**: Adds learnable virtual tokens to guide the model. Lightweight and modular, especially useful when you want to adapt the model to multiple tasks without touching the original weights.
- **AdaLoRA**: Dynamic LoRA variant that adjusts the magnitude of updates during training. Combines efficiency with adaptive scaling, achieving strong performance with controlled parameter growth.

**Observation:**  
All three PEFTs drastically reduce trainable parameters compared to full fine-tuning while maintaining comparable accuracy — demonstrating the effectiveness of parameter-efficient fine-tuning for stance classification tasks.
